### Functions

In [1]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_distance_matrix(locs):
    """Creates distance matrix for the problem."""
    distance_matrix = []
    matrix_len = len(locs)+1
    for i,j in locs:
        n=0
        row = [0]*matrix_len
        for k,l in locs:
            row[n] = abs(i-k) + abs(j-l)
            n+=1
        distance_matrix.append(row)
    row = [0]*matrix_len
    distance_matrix.append(row)
    return distance_matrix


def create_pickups_deliveries(unique_vehicle_locs, pickup_locs):
    """Creates pickup and delivery pairs. """
    pickups_deliveries = []
    for i in range(len(pickup_locs)):
        pickups_deliveries.append((len(unique_vehicle_locs)+i, len(unique_vehicle_locs)+len(pickup_locs)+i))
    return pickups_deliveries


def create_data_model(path="inputFile.txt"):
    """Preprocess data for the problem."""
    with open(path, 'r') as file:
        content = file.read()

    vehicle_content = content.split("\n\n")[0]
    delivery_content = content.split("\n\n")[1]
    data = {}
    vehicle_caps = []
    vehicle_locs = []
    time_windows = []
    pickup_locs = []
    delivery_locs = []
    delivery_ids = []
    
    for i in vehicle_content.split("\n"):
        vehicle_loc = []
        idx = 0
        for j in i.split(','):
            if idx == 1:
                vehicle_caps.append(int(j))
            elif idx > 1:
                vehicle_loc.append(int(j))
            idx += 1
        vehicle_locs.append(tuple(vehicle_loc))
    

    for i in delivery_content.split("\n"):
        pickup_loc = []
        delivery_loc = []
        idx = 0
        for j in i.split(','):
            if idx == 0:
                delivery_ids.append(int(j))
            elif idx == 1:
                time_windows.append(int(j))
            elif idx > 1 and idx < 4:
                pickup_loc.append(int(j))
            else:
                delivery_loc.append(int(j))
            idx += 1

        pickup_locs.append(tuple(pickup_loc))
        delivery_locs.append(tuple(delivery_loc))    


    unique_vehicle_locs = list(set(vehicle_locs))
    num_vehicles = len(vehicle_locs)
    starts = [unique_vehicle_locs.index(loc) for loc in vehicle_locs]
    ends = [len(unique_vehicle_locs) + len(pickup_locs) + len(delivery_locs)] * num_vehicles
    locs = unique_vehicle_locs + pickup_locs + delivery_locs
    demands = [0]*(len(locs)+1)
    demands[-(len(delivery_locs)+1):-1] = [1]*len(delivery_locs)
    distance_matrix = create_distance_matrix(locs)
    pickups_deliveries = create_pickups_deliveries(unique_vehicle_locs, pickup_locs)
    time_windows = {i[1]: tw for i, tw in zip(pickups_deliveries, time_windows)}
    names = {}
    for i in range(len(pickups_deliveries)):
        names[pickups_deliveries[i][0]] = f'p{delivery_ids[i]}'
        names[pickups_deliveries[i][1]] = f'd{delivery_ids[i]}'

    data['distance_matrix'] = distance_matrix
    data['pickups_deliveries'] = pickups_deliveries
    data["num_vehicles"] = num_vehicles
    data["starts"] = starts
    data["ends"] = ends
    data["vehicle_capacities"] = vehicle_caps
    data["demands"] = demands
    data['time_windows'] = time_windows
    data['names'] = names
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}\n")
    total_distance = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id+1}:\n"
        route_distance = 0
        while not routing.IsEnd(index):
            if manager.IndexToNode(index) in data['names'].keys():
                plan_output += f" {data['names'][manager.IndexToNode(index)]} -> "
            else:
                plan_output += f" {manager.IndexToNode(index)} -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f"{manager.IndexToNode(index)}\n"
        plan_output += f"Distance (duration) of the route: {route_distance}m\n"
        print(plan_output)
        total_distance += route_distance
    print(f"Total Distance (duration) of all routes: {total_distance}m")


def main(path="inputFile.txt"):
    """Entry point of the program."""

    # Instantiate the data problem.
    data = create_data_model(path)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data['starts'], data['ends']
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Define cost of each arc.
    def distance_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = "Distance"
    routing.AddDimension(
        transit_callback_index,
        0,      # no slack
        15000,  # vehicle maximum travel distance
        True,   # start cumul to zero
        dimension_name,
    )
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    #distance_dimension.SetGlobalSpanCostCoefficient(100)


    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,                           # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,                        # start cumul to zero
        "Capacity",
    )

    # Define pickup and delivery constraints.
    for request in data["pickups_deliveries"]:
        pickup_index = manager.NodeToIndex(request[0])
        delivery_index = manager.NodeToIndex(request[1])
        routing.AddPickupAndDelivery(pickup_index, delivery_index)
        routing.solver().Add(
            routing.VehicleVar(pickup_index) == routing.VehicleVar(delivery_index)
        )
        routing.solver().Add(
            distance_dimension.CumulVar(pickup_index)
            <= distance_dimension.CumulVar(delivery_index)
        )


    # Time penalties
    restricted_nodes = data['time_windows']  
    penalty_per_minute = 10
    for node, max_time in restricted_nodes.items():
        node_index = manager.NodeToIndex(node)
        distance_dimension.SetCumulVarSoftUpperBound(node_index, max_time, penalty_per_minute)


    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.use_depth_first_search = True
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION
    )

    # search_parameters.time_limit.seconds = 60

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)

### Example Sol.

In [2]:
main("example.txt")

Objective: 30

Route for vehicle 1:
 0 ->  p13 ->  p14 ->  d14 ->  d13 -> 10
Distance (duration) of the route: 7m

Route for vehicle 2:
 0 ->  p11 ->  p12 ->  d11 ->  d12 -> 10
Distance (duration) of the route: 13m

Route for vehicle 3:
 1 -> 10
Distance (duration) of the route: 0m

Total Distance (duration) of all routes: 20m


### InputFile Sol.

In [3]:
main("inputFile.txt")

Objective: 388

Route for vehicle 1:
 1 ->  p22 ->  p23 ->  p24 ->  p25 ->  d25 ->  d24 ->  d23 ->  d22 -> 50
Distance (duration) of the route: 7m

Route for vehicle 2:
 2 ->  p20 ->  p21 ->  d21 ->  d20 -> 50
Distance (duration) of the route: 16m

Route for vehicle 3:
 1 ->  p29 ->  p28 ->  p27 ->  p26 ->  d29 ->  d28 ->  d27 ->  d26 -> 50
Distance (duration) of the route: 13m

Route for vehicle 4:
 3 ->  p30 ->  d30 -> 50
Distance (duration) of the route: 8m

Route for vehicle 5:
 0 ->  p19 ->  p18 ->  p17 ->  p14 ->  p15 ->  p16 ->  d17 ->  d19 ->  d18 ->  d16 ->  d15 ->  d14 -> 50
Distance (duration) of the route: 25m

Route for vehicle 6:
 1 ->  p31 ->  p11 ->  p13 ->  p12 ->  p33 ->  p32 ->  d31 ->  d33 ->  d32 ->  d11 ->  d12 ->  d13 -> 50
Distance (duration) of the route: 19m

Total Distance (duration) of all routes: 88m
